In [64]:
from neo4j import GraphDatabase

In [65]:
# connexion à la BDD
uri             = "bolt://localhost:7687"
userName        = "neo4j"
password        = "neo4j"


graphDB_Driver = GraphDatabase.driver(uri, auth=(userName, password))

In [66]:
def execute_query(query):
    """
    Fonction de base exécutant les requêtes passées en paramètre
    """
    with graphDB_Driver.session() as session:
        result = session.run(query)
        return result.data()


def get_all_nodes():
    """
    Récupère tous les nœuds du graphe connecté
    """
    return execute_query("MATCH (n) RETURN n")

print(get_all_nodes())
    

[{'n': {'tagline': 'Welcome to the Real World', 'title': 'The Matrix', 'released': 1999}}, {'n': {'born': 1964, 'name': 'Keanu Reeves'}}, {'n': {'born': 1967, 'name': 'Carrie-Anne Moss'}}, {'n': {'born': 1961, 'name': 'Laurence Fishburne'}}, {'n': {'born': 1960, 'name': 'Hugo Weaving'}}, {'n': {'born': 1967, 'name': 'Lilly Wachowski'}}, {'n': {'born': 1965, 'name': 'Lana Wachowski'}}, {'n': {'born': 1952, 'name': 'Joel Silver'}}, {'n': {'born': 1978, 'name': 'Emil Eifrem'}}, {'n': {'tagline': 'Free your mind', 'title': 'The Matrix Reloaded', 'released': 2003}}, {'n': {'tagline': 'Everything that has a beginning has an end', 'title': 'The Matrix Revolutions', 'released': 2003}}, {'n': {'tagline': 'Evil has its winning ways', 'title': "The Devil's Advocate", 'released': 1997}}, {'n': {'born': 1975, 'name': 'Charlize Theron'}}, {'n': {'born': 1940, 'name': 'Al Pacino'}}, {'n': {'born': 1944, 'name': 'Taylor Hackford'}}, {'n': {'tagline': "In the heart of the nation's capital, in a courtho

In [67]:
def get_node_by_id(id):
    """
    Retourne un nœud selon son id
    """
    return execute_query(f"MATCH (n) WHERE id(n) = {id} RETURN n")

print(get_node_by_id(87))

[{'n': {'tagline': 'Pain heals, Chicks dig scars... Glory lasts forever', 'title': 'The Replacements', 'released': 2000}}]


In [68]:
def get_all_relationships():
    """
    Renvoi toutes les relations d'un graphe
    """
    with graphDB_Driver.session() as session:
        result = session.run("MATCH ()-[r]->() RETURN r")
        return [record['r'] for record in result]

print(get_all_relationships())

[<Relationship element_id='5:04fde3f0-35b3-4c2a-8f37-503323d6acc3:0' nodes=(<Node element_id='4:04fde3f0-35b3-4c2a-8f37-503323d6acc3:1' labels=frozenset() properties={}>, <Node element_id='4:04fde3f0-35b3-4c2a-8f37-503323d6acc3:0' labels=frozenset() properties={}>) type='ACTED_IN' properties={'roles': ['Neo']}>, <Relationship element_id='5:04fde3f0-35b3-4c2a-8f37-503323d6acc3:1' nodes=(<Node element_id='4:04fde3f0-35b3-4c2a-8f37-503323d6acc3:2' labels=frozenset() properties={}>, <Node element_id='4:04fde3f0-35b3-4c2a-8f37-503323d6acc3:0' labels=frozenset() properties={}>) type='ACTED_IN' properties={'roles': ['Trinity']}>, <Relationship element_id='5:04fde3f0-35b3-4c2a-8f37-503323d6acc3:2' nodes=(<Node element_id='4:04fde3f0-35b3-4c2a-8f37-503323d6acc3:3' labels=frozenset() properties={}>, <Node element_id='4:04fde3f0-35b3-4c2a-8f37-503323d6acc3:0' labels=frozenset() properties={}>) type='ACTED_IN' properties={'roles': ['Morpheus']}>, <Relationship element_id='5:04fde3f0-35b3-4c2a-8f37

In [69]:
def get_relationship_by_id(relationship_id):
    """
    Renvoi la relation correspondant à l'ID donnée
    """
    with (graphDB_Driver.session() as session):
        result = session.run("MATCH ()-[r]->() WHERE id(r) = $relationshipId RETURN r",relationshipId=relationship_id)
        return result.single()['r']
    
print(get_relationship_by_id(14))

<Relationship element_id='5:04fde3f0-35b3-4c2a-8f37-503323d6acc3:14' nodes=(<Node element_id='4:04fde3f0-35b3-4c2a-8f37-503323d6acc3:7' labels=frozenset() properties={}>, <Node element_id='4:04fde3f0-35b3-4c2a-8f37-503323d6acc3:9' labels=frozenset() properties={}>) type='PRODUCED' properties={}>


In [70]:
def create_node(label, properties):
    """
    Crée une node
    """
    with graphDB_Driver.session() as session:
        query = (
            f"CREATE (n:{label} $props) "
            "RETURN n"
        )
        result = session.run(query, props=properties)
        return result.single()['n']
    
label = "Movie"
properties = {"tagline": "Once upon another time...",
              "title": "Shrek 3",
              "released": 2007}


# created_node = create_node(label, properties)

In [72]:
def create_relationship(start_node_id, end_node_id, rel_type, properties):
    """
    Créé une relation entre deux nodes
    """
    with graphDB_Driver.session() as session:
        query = (
            f"MATCH (start), (end) "
            f"WHERE id(start) = {start_node_id} AND id(end) = {end_node_id} "
            f"CREATE (start)-[r:{rel_type} $props]->(end) "
            "RETURN r"
        )
        result = session.run(query, props=properties)
        return result.single()['r']
    
start_node_id = 0
end_node_id = 1
rel_type = "ACTED_IN"
properties = {"role": "Neo"}
# created_relationship = create_relationship(start_node_id, end_node_id, rel_type, properties)